# The General Claims Data
- Primary question of interest: Who was at fault?
<br> <br>
- Secondary questions of interest:
    - What Claim Group should the claim be routed to?
    - Can we predict severity type (minor, moderate, major, life threatening, death)
    - What was the cause of the loss?

In [1]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg

In [2]:
import nltk
from nltk import word_tokenize

In [3]:
from nltk.corpus import stopwords
# stop = stopwords.words('english')

In [4]:
stop = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 
 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 
 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 
 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 
 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 
 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 
 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'doing', 'it', 'how', 'further', 'was', 
 'here', 'than']

In [5]:
import gensim

In [6]:
# general_tokens_df = pd.read_csv('data/general_with_tokens.csv')

In [7]:
# general_tokens_df.head()

In [8]:
from ast import literal_eval

In [9]:
# def join_strings(df, columns):
#     n_df = df
#     for col in columns:
#         def join_col(row):
#             claim_list = row[col]
#             return ' '.join(claim_list)
#         n_df[col] = n_df.apply(join_col, axis = 1)
#     return n_df
# convert lists from strings back to lists
def convert_to_list(df,cols):
    n_df = df
    for col in cols:
        def de_list(row):
            list_str = row[col]
            return literal_eval(list_str)
        n_df[col] = n_df.apply(de_list, axis = 1)
    return n_df

In [10]:
# general_tokens_df = convert_to_list(general_tokens_df, ['claim_level_list', 'exposure_level_list',
#                                                        'accident_desc_list', 'injury_desc_list',
#                                                        'damage_desc_list'])

In [11]:
general_df = pd.read_csv('data/general_with_word_counts.csv')


In [12]:
general_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,...,insd_count,insured_count,rear_count,deployed_count,towed_count,totaled_count,head_count,ambulance_count,concussion_count,whiplash_count
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,...,0,0,0,0,0,0,0,0,0,0
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,...,0,2,0,0,0,0,0,0,0,0
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,...,4,4,1,0,0,0,0,0,0,0
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,...,4,4,1,0,0,0,0,0,0,0
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,...,0,10,0,0,0,0,0,0,0,0


In [13]:
general_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119272 entries, 0 to 119271
Data columns (total 47 columns):
ClaimID_RGEN                      119272 non-null int64
ExposureID_RGEN                   119272 non-null int64
ClaimLevelBody                    119272 non-null object
ExposureLevelBody                 119272 non-null object
AccidentDescription               119272 non-null object
InjuryDescription                 119272 non-null object
DamageDescription                 119272 non-null object
SeverityTypeName                  86721 non-null object
ClaimGroup                        119257 non-null object
FaultRatingName                   119271 non-null object
LossCauseName                     119272 non-null object
CCCreateTime                      119272 non-null object
CloseDate                         99601 non-null object
StateName                         119272 non-null object
ClaimLevelBody_preprocess         119272 non-null object
ExposureLevelBody_preprocess      1192

In [14]:
# general_no_unknown = general_df.loc[general_df.loc[:,'FaultRatingName'] != 'Fault unknown',:].reset_index(drop=True)

In [15]:
general_df = general_df.fillna({'exposure_level_list':' ', 'injury_desc_list':' ', 'damage_desc_list': ' '})

In [16]:
# general_no_unknown.info()

In [17]:
nlp = en_core_web_lg.load()

In [18]:
def column_to_list_text(df, col_name):
    return df[col_name].tolist()

In [19]:
def lower_columns(df, col_list):
    n_df = df
    for col_name in col_list:
        n_col = col_name + '_preprocess'
        n_df[n_col] = n_df[col_name].str.lower()
    return n_df

In [20]:
# lowered_df = lower_columns(general_df, ['ClaimLevelBody', 'ExposureLevelBody', 'AccidentDescription', 
#                                         'InjuryDescription', 'DamageDescription'])

In [21]:
# lowered_df.head()

In [22]:
def tokenize_claim_level(row):
#     list_cols = [['ClaimLevelBody_preprocess', 'ExposureLevelBody_preprocess', 'AccidentDescription_preprocess', 
#                                         'InjuryDescription_preprocess', 'DamageDescription_preprocess']]
    text = row['ClaimLevelBody_preprocess']
    text_tokenize = [token for token in word_tokenize(text) if token.isalpha() and token not in stop]
    return text_tokenize
        
def pre_process(df, col_list):
    for col in col_list:
        df[col] = df.apply(lambda x: tokenize(x, col), axis=1)
    return df

In [23]:
#lowered_df['ClaimLevelBody_preprocess'] = lowered_df.apply(tokenize_claim_level, axis = 1)

In [24]:
#out_df.head()

In [25]:
general_df.claim_level_list[0]

'reviewed coverage sc rated policy loss effective dates coverage ni operating vnop vehicle appears loaner veh policy set cx non payment reinstated without lapse send ror citing definition covered auto insuring agreement section liability coverage liability modified compneg state need statements police report complete lia investigation based report loss driver pulled nis row exposure driver appears allstate lets confirm rule umpdumbi exposure appears pd rule injuries clmnt driver determine severity nis injuries secure uim rejection form action plan send ror complete vnop investigation statements confirm ownership veh insurance infomation vnop confirm driver lia coverage rule umpdumbi complete liability investigation statementspolice reportset appraisal needed rule serious injuries secure uim rejection form needed recovery applicable reviewed coverage sc rated policy loss effective dates coverage ni operating vnop vehicle appears loaner veh policy set cx non payment reinstated without la

In [26]:
# lowered_df.head()

In [27]:
fault_ratings = column_to_list_text(general_df, 'FaultRatingName')

In [28]:
claim_level_body = column_to_list_text(general_df, 'claim_level_list')

In [29]:
exposure_level_body = column_to_list_text(general_df, 'exposure_level_list')

In [30]:
accident_description = column_to_list_text(general_df, 'accident_desc_list')

In [31]:
injury_description = column_to_list_text(general_df, 'injury_desc_list')

In [32]:
damage_description = column_to_list_text(general_df, 'damage_desc_list')

In [33]:
# claim_level_tokenized = [[token for token in word_tokenize(doc)] 
#                            for doc in claim_level_body]
# exposure_level_tokenized = [[token for token in word_tokenize(doc)] 
#                            for doc in exposure_level_body]
# accident_desc_tokenized = [[token for token in word_tokenize(doc)] 
#                            for doc in accident_description]
# injury_desc_tokenized = [[token for token in word_tokenize(doc)] 
#                            for doc in injury_description]
# damage_desc_tokenized = [[token for token in word_tokenize(doc) ] 
#                          for doc in damage_description]

# [token for token in word_tokenize(damage_description) if token.isalpha()]

In [34]:
from collections import Counter

In [35]:
# exposure_level_tokenized[4]

In [36]:
# word_counts_claim = [dict(Counter(doc)) for doc in claim_level_tokenized]
# word_counts_exposure = [dict(Counter(doc)) for doc in exposure_level_tokenized]
# word_counts_accident = [dict(Counter(doc)) for doc in accident_desc_tokenized]
# word_counts_injury = [dict(Counter(doc)) for doc in injury_desc_tokenized]
# word_counts_damage = [dict(Counter(doc)) for doc in damage_desc_tokenized]

In [37]:
# word_counts_accident[30052]

In [38]:
# claim_word_counts = [len(doc) for doc in claim_level_tokenized]
# exposure_word_counts = [len(doc) for doc in exposure_level_tokenized]
# accident_word_counts = [len(doc) for doc in accident_desc_tokenized]
# injury_word_counts = [len(doc) for doc in injury_desc_tokenized]
# damage_word_counts = [len(doc) for doc in damage_desc_tokenized]


In [40]:
# iv_list = [0] * len(word_counts_claim)
# liability_list = [0] * len(word_counts_claim)
# dcv_list = [0] * len(word_counts_claim)
# inj_list = [0] * len(word_counts_claim)
# vnop_list = [0] * len(word_counts_claim)
# ac_list = [0] * len(word_counts_claim)
# cv_list = [0] * len(word_counts_claim)
# div_list = [0] * len(word_counts_claim)
# insd_list = [0] * len(word_counts_claim)
# insured_list = [0] * len(word_counts_claim)
# clmt_list = [0] * len(word_counts_claim)

In [42]:
# head_list = [0] * len(word_counts_injury)
# ambulance_list = [0] * len(word_counts_injury)
# concussion_list = [0] * len(word_counts_injury)
# whiplash_list = [0] * len(word_counts_injury)

In [43]:
# rear_list = [0] * len(word_counts_accident)

In [44]:
# deployed_list = [0] * len(word_counts_damage)
# towed_list = [0] * len(word_counts_damage)
# totaled_list = [0] * len(word_counts_damage)

In [45]:
# for ind, count_dict in enumerate(word_counts_accident):
#     if 'rear' in count_dict:
#         rear_list[ind] = count_dict['rear']

In [46]:
# for ind, count_dict in enumerate(word_counts_damage):
#     if 'deployed' in count_dict:
#         deployed_list[ind] = count_dict['deployed']
#     if 'towed' in count_dict:
#         towed_list[ind] = count_dict['towed']
#     if 'totaled' in count_dict:
#         totaled_list[ind] = count_dict['totaled']

In [47]:
# for ind, count_dict in enumerate(word_counts_injury):
#     if 'head' in count_dict:
#         head_list[ind] = count_dict['head']
#     if 'ambulance' in count_dict:
#         ambulance_list[ind] = count_dict['ambulance']
#     if 'concussion' in count_dict:
#         concussion_list[ind] = count_dict['concussion']
#     if 'whiplash' in count_dict:
#         whiplash_list[ind] = count_dict['whiplash']

In [48]:
# for ind, count_dict in enumerate(word_counts_claim):
#     if 'iv' in count_dict:
#         iv_list[ind] = count_dict['iv']
#     else:
#         iv_list[ind] = 0
#     if 'liability' in count_dict:
#         liability_list[ind] = count_dict['liability']
#     else:
#         liability_list[ind] = 0
#     if 'dcv' in count_dict:
#         dcv_list[ind] = count_dict['dcv']
#     else:
#         dcv_list[ind] = 0
#     if 'inj' in count_dict:
#         inj_list[ind] = count_dict['inj']
#     else:
#         inj_list[ind] = 0
#     if 'vnop' in count_dict:
#         vnop_list[ind] = count_dict['vnop']
#     else:
#         vnop_list[ind] = 0
#     if 'ac' in count_dict:
#         ac_list[ind] = count_dict['ac']
#     else:
#         ac_list[ind] = 0
#     if 'cv' in count_dict:
#         cv_list[ind] = count_dict['cv']
#     else:
#         cv_list[ind] = 0
#     if 'div' in count_dict:
#         div_list[ind] = count_dict['div']
#     else:
#         div_list[ind] = 0
#     if 'insd' in count_dict:
#         insd_list[ind] = count_dict['insd']
#     else:
#         insd_list[ind] = 0
#     if 'insured' in count_dict:
#         insured_list[ind] = count_dict['insured']
#     else:
#         insured_list[ind] = 0

In [49]:
# iv_list[:10]

In [50]:
# word_counts_claim[2]

In [51]:
# word_cols = {'claim_level_word_count': claim_word_counts,
#              'exposure_word_count': exposure_word_counts,
#              'accident_desc_word_count': accident_word_counts,
#     'injury_desc_word_count': injury_word_counts,
#              'damage_desc_word_count': damage_word_counts,
#     'iv_count': iv_list,
#          'liability_count': liability_list,
#          'dcv_count': dcv_list,
#          'inj_count': inj_list,
#              'vnop_count': vnop_list,
#             'ac_count': ac_list,
#             'cv_count': cv_list,
#             'div_count': div_list,
#             'insd_count': insd_list,
#             'insured_count': insured_list,
#             'rear_count': rear_list,
#             'deployed_count': deployed_list,
#             'towed_count': towed_list,
#             'totaled_count': totaled_list,
#             'head_count': head_list,
#             'ambulance_count': ambulance_list,
#             'concussion_count': concussion_list,
#             'whiplash_count': whiplash_list}
# df_spec_word_count = pd.DataFrame.from_dict(word_cols)

In [52]:
# df_spec_word_count.head()

In [53]:
# dict_cols = {'word_counts_claim': word_counts_claim,
#          'word_counts_exposure': word_counts_exposure,
#          'word_counts_accident': word_counts_accident,
#          'word_counts_injury': word_counts_injury,
#              'word_counts_damage': word_counts_damage}
# df_word_count = pd.DataFrame.from_dict(dict_cols)

In [54]:
# df_word_count.head()

In [56]:
# token_cols = {'claim_level_list': claim_level_tokenized,
#          'exposure_level_list': exposure_level_tokenized,
#          'accident_desc_list': accident_desc_tokenized,
#          'injury_desc_list': injury_desc_tokenized,
#              'damage_desc_list': damage_desc_tokenized}
# df = pd.DataFrame.from_dict(token_cols)

In [57]:
# df.head()

In [58]:
# len(df)

In [59]:
# df_spec_word_counts = pd.concat([df_with_counts, df_spec_word_count], axis = 1)

In [60]:
# df_with_tokens = pd.concat([lowered_df, df], axis = 1)

In [61]:
# df_with_counts = pd.concat([general_df, df_spec_word_count], axis = 1)

In [62]:
general_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,...,insd_count,insured_count,rear_count,deployed_count,towed_count,totaled_count,head_count,ambulance_count,concussion_count,whiplash_count
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,...,0,0,0,0,0,0,0,0,0,0
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,...,0,2,0,0,0,0,0,0,0,0
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,...,4,4,1,0,0,0,0,0,0,0
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,...,4,4,1,0,0,0,0,0,0,0
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,...,0,10,0,0,0,0,0,0,0,0


In [63]:
# len(df_with_tokens)

In [64]:
# df_with_counts.to_csv('data/general_with_word_counts.csv', index = False)

In [65]:
# lowered_df.FaultRatingName.value_counts()


In [66]:
# general_tokens_df.iloc[0,22]

In [67]:
def join_strings(df, columns):
    n_df = df
    for col in columns:
        def join_col(row):
            claim_list = row[col]
            return ' '.join(claim_list)
        n_df[col] = n_df.apply(join_col, axis = 1)
    return n_df

In [68]:
# general_full_strings = join_strings(df_with_tokens, ['claim_level_list', 'exposure_level_list',
#                                                        'accident_desc_list', 'injury_desc_list',
#                                                        'damage_desc_list'])

In [69]:
# general_full_strings.head()

In [119]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [120]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

In [121]:
X_train, X_test, y_train, y_test = train_test_split(accident_description, fault_ratings, 
                                                    test_size=0.25, random_state=42)

In [122]:
model = text_clf.fit(X_train, y_train)

In [123]:
predicted_MNB = text_clf.predict(X_test)

In [124]:
predicted_MNB[:5]

array(['Insured at fault', 'Insured at fault', 'Insured at fault',
       'Insured at fault', 'Insured at fault'], dtype='<U22')

In [125]:
np.mean(predicted_MNB == y_test)

0.6397142665504058

In [126]:
len(predicted_MNB)

29818

In [127]:
text_clf.score(X_test, y_test)

0.6397142665504058

In [128]:
from collections import Counter

In [129]:
Counter(predicted_MNB)

Counter({'Insured at fault': 28402,
         'Other party at fault': 912,
         'No fault': 404,
         'Comparative Negligence': 100})

In [98]:
28402/29818

0.9525119055603998

In [99]:
74304/len(general_no_unknown)

0.622979408411027